# Rabi Oscillations

This notebook conducts several experiments which measure the cavity field with time. For each experiment, the amplitude of a the Gaussian driving pulse is varied. This induces Rabi-Oscillations in the system. By measuring the response of the cavity field, we can compute the qubit excited state population $p_e$ as done by Bianchetti et. al. This is done in a related MATLAB file.

## 1. Preliminaries

In [1]:
from qiskit.tools.jupyter import *
from qiskit import IBMQ

import numpy as np
import matplotlib.pyplot as plt

# do this so that when call backend, will look nice, report properties

%config InlineBackend.figure_format = 'svg' # Makes the images look nice

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support OpenPulse"

# check qiskit version
import qiskit
qiskit.__qiskit_version__

import warnings
warnings.filterwarnings('ignore')

In [2]:
# basic unit conversion factors and device properties

# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds
scale_factor = 1e-14 # scale factor to remove factors of 10 from the data

# basic parameters
qubit = 0 # there is only one qubit
dt = backend_config.dt
#defaults
backend_defaults = backend.defaults()

# drive parameters
[[min_drive_freq, max_drive_freq]] = backend_config.qubit_lo_range # range of freqs for qubit drive, 1Ghz around wq
[[min_meas_freq, max_meas_freq]] = backend_config.meas_lo_range # range of freqs for measurement drive

# cavity parameters
cavity_freq = backend_defaults.meas_freq_est[qubit]

# qubit parameters
qubit_props_dict = backend.properties().qubit_property(0)
qubit_freq = qubit_props_dict['frequency'][0]
# qubit_freq = backend_defaults.qubit_freq_est[qubit] #same result
qubit_T1 = qubit_props_dict['T1'][0]
qubit_T2 = qubit_props_dict['T2'][0]

print(f"sampling time: {dt / ns : .2f} ns.")
print(f"qubit frequency: {qubit_freq / GHz} GHz.")
print(f"qubit decay: {1 /(qubit_T1* GHz)} GHz.")
print(f"cavity frequency: {cavity_freq / GHz} GHz.")

print(f"qubit T1: {qubit_T1/us} us.")

sampling time:  0.22 ns.
qubit frequency: 4.974296454880922 GHz.
qubit decay: 6.69437024702333e-06 GHz.
cavity frequency: 6.993427855 GHz.
qubit T1: 149.37924899577413 us.


In [ ]:
# defaults and other parameters used to define control pulses
from qiskit import pulse
from qiskit.pulse import pulse_lib
from qiskit.pulse import Play # comment out if qiskit-terra vesrion less than 0.13.0

# samples need to be multiples of 16
def get_closest_multiple_of_16(num):
    return int(num + 8 ) - (int(num + 8 ) % 16)


# Find out which group of qubits need to be acquired with this qubit
meas_map_idx = None
for i, measure_group in enumerate(backend_config.meas_map):
    if qubit in measure_group:
        meas_map_idx = i
        break
assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"
print(f"Qubit in measurement group {meas_map_idx}")
qubit_meas_group = backend_config.meas_map[meas_map_idx]

#default instructions
inst_sched_map = backend_defaults.instruction_schedule_map #default pulses so dont have to construct by hand

drive_chan = pulse.DriveChannel(qubit)
meas_chan = pulse.MeasureChannel(qubit)
acq_chan = pulse.AcquireChannel(qubit)

## 2. Pulse Schedules

The pulse schedules are defined just as they are in the Cavity Dynamics notebook. The only difference is that this notebook doesn't require seperate jobs for the ground and excited states. Instead, it defines a Gaussian pulse, with a user-defined amplitude to drive the qubit.

In [4]:
# acqSamples_us         : Duration of the acquisition window, in us
# cavityPumpDuration_us : List of times delays (in us) between the start of the measurement and when the cavity field is acquired.
# measAmp               : Strength of the measurement drive
# measSigma_us          : Width of the Gaussian part of the measurement Pulse, in us
# measRiseFall_us       : Rise and fall time of the Gaussian part of the measurement Pulse, in us
# driveAmp              : Strength of the Rabi-Pulse drive
# driveSigma_us         : Width of the Rabi-Pulse drive in us
def generateSchedules(acqSamples_us, cavityPumpDuration_us, measAmp, measSigma_us, measRiseFall_us, driveAmp, driveSigma_us):
        
    # convert all the above durations into samples (in units of dt)
    # then quantize the samples to the closest multiple of 16
    cavityPumpSamples = [get_closest_multiple_of_16(cavityPumpTime_us * us/dt) for cavityPumpTime_us in cavityPumpDuration_us]
    acqSamples = get_closest_multiple_of_16(acqSamples_us * us/dt)
    measSigma = get_closest_multiple_of_16(measSigma_us * us/dt)       
    measRiseFall = get_closest_multiple_of_16(measRiseFall_us * us/dt) 

    ########################### Construct the Rabi-Pulse ###########################
    driveSamples_us = driveSigma_us*8
    driveSigma = get_closest_multiple_of_16(driveSigma_us * us /dt)
    driveSamples = get_closest_multiple_of_16(driveSamples_us * us /dt) 
    
    rabi_pulse = pulse_lib.gaussian(duration=driveSamples, amp=driveAmp, 
                                    sigma=driveSigma, name=f"Rabi drive amplitude = {driveAmp}")
    
    ######################## Acquisiton portion of schedule ########################
    meas_pulse = pulse_lib.gaussian_square(duration=acqSamples, 
                                   sigma=measSigma,
                                   amp=measAmp,
                                   risefall=measRiseFall,
                                   name='measurement_pulse')
    # apply meas_pulse to measurement channel
    measure_schedule = Play(meas_pulse, meas_chan) 
    # this acquire object is how add variable duration acquisition windows turns on acquisition channel
    measure_schedule += pulse.Acquire(duration=acqSamples, 
                                      channel=[pulse.AcquireChannel(i) for i in qubit_meas_group],
                                      mem_slot=[pulse.MemorySlot(i) for i in qubit_meas_group])

    ############### Make the schedule for rabi-pulse preparation ##################    
    schedules = []
    
    # iterate through each cavity pump duration
    for cavityPumpSample in cavityPumpSamples: 
        # create an empty schedule
        schedule = pulse.Schedule(name=f"|e> readout dynamics t={cavityPumpSample * dt/us} us")
        
        # add the pi-pulse
        schedule += rabi_pulse(drive_chan)
        
        # construct a measurement pulse with duration cavityPumpSample
        cavity_pulse = pulse_lib.gaussian_square(duration=cavityPumpSample,
                                   sigma=measSigma,
                                   amp=measAmp,
                                   risefall=measRiseFall,
                                   name='cavity_pump_pulse')
        
        # apply the measurement pulse to the measurement channel; delay this to after the pi-pulse is completed
        schedule += Play(cavity_pulse, meas_chan) << schedule.duration
        
        # add the acquisition schedule (as defined above) immediately after the the measurement tone
        schedule += measure_schedule
        
        # append to the list of schedules for the excited state
        schedules.append(schedule)
    
    
    return schedules

### Assemble and Run the Job

In [5]:
from qiskit import assemble
from qiskit.tools.monitor import job_monitor

# driveFreq      : Frequency of the Driving (in Hz)
# measFreq       : Frequency of the Measurement (in Hz)
# numShotsPerFreq: Number of experiments per frequency
# schedules      : List of schedules for the Rabi-experiment
def runJob(driveFreq, measFreq, numShotsPerFreq, schedules):
    # Match the frequencies to the channels
    channel_freqs = {drive_chan: driveFreq, meas_chan: measFreq}
    
    # Assemble the jobs 
    acquire_sweep_program = assemble(schedules,
                                     backend=backend, 
                                     meas_level=1,
                                     meas_return='avg',
                                     shots=numShotsPerFreq,
                                     schedule_los=[channel_freqs] * len(schedules))

    # Run the jobs
    job = backend.run(acquire_sweep_program)
    print('rabi sweep: ' + job.job_id())

    return job.job_id()

## 3. The Rabi Experiment

In [6]:
def cavityDynamics(startTime_us, stopTime_us, timeStep_us, acqSamples_us, 
                   measAmp, measSigma_us, measRiseFall_us, measFreq,
                   driveAmp, driveFreq, driveSigma_us,
                   numShotsPerFreq):
    
    readout_values = []
    
    cavityPumpTimes_us = np.arange(startTime_us, stopTime_us, timeStep_us)
    print("\nRunning Cavity Dynamics from {0}us to {1}us, in steps of {2}us".format(cavityPumpTimes_us[0], 
                                                                                    cavityPumpTimes_us[-1], 
                                                                                    timeStep_us))
    print("Drive Amplitude = {0}".format(driveAmp))               
                                                                                    
    schedules = generateSchedules(acqSamples_us, cavityPumpTimes_us, 
                                   measAmp, measSigma_us, measRiseFall_us, 
                                   driveAmp, driveSigma_us)
    
    jobID = runJob(driveFreq, measFreq, numShotsPerFreq, schedules)

    job = backend.retrieve_job(jobID)

    job_monitor(job)
                                                                                    
    print("Errors: {0}".format(job.error_message()))

    # timeout parameter set to 120 seconds
    acquire_sweep_results = job.result(timeout=120)

    # load complex measurement results, scale
    for i in range(len(cavityPumpTimes_us)):
        readout_values.append(acquire_sweep_results.get_memory(i)[qubit]*scale_factor)
    
    return readout_values

Run the Rabi Experiment  

In [7]:
# Rabi experiment parameters
numRabiPoints = 50

# Drive amplitude values to iterate over: 20 amplitudes evenly spaced from 0 to 0.75
driveAmp_min = 0
driveAmp_max = 0.75
driveAmps = np.linspace(driveAmp_min, driveAmp_max, numRabiPoints)

In [8]:
chi = 0.421 * MHz
measFreq = cavity_freq
driveFreq = qubit_freq - chi

In [9]:
all_readouts = []
for driveAmp in driveAmps:
    readout_vals = cavityDynamics(startTime_us = 0.1, stopTime_us = 5.0, timeStep_us = 0.25, acqSamples_us = 0.05, 
                                  measAmp = 0.5, measSigma_us = 0.01, measRiseFall_us = 0.001, measFreq = measFreq,
                                  driveAmp = driveAmp, driveFreq = driveFreq, driveSigma_us = 0.075,
                                  numShotsPerFreq = 1024)
    all_readouts.append(readout_vals)


Running Cavity Dynamics from 0.1us to 4.849999999999999us, in steps of 0.25us
Drive Amplitude = 0.0
rabi sweep: 5ec135d683f02c001a45ec3a
Job Status: job has successfully run
Errors: None

Running Cavity Dynamics from 0.1us to 4.849999999999999us, in steps of 0.25us
Drive Amplitude = 0.015306122448979591
rabi sweep: 5ec136f0b86619001bbebb71
Job Status: job has successfully run
Errors: None

Running Cavity Dynamics from 0.1us to 4.849999999999999us, in steps of 0.25us
Drive Amplitude = 0.030612244897959183
rabi sweep: 5ec13866927380001bb81b38
Job Status: job has successfully run
Errors: None

Running Cavity Dynamics from 0.1us to 4.849999999999999us, in steps of 0.25us
Drive Amplitude = 0.04591836734693877
rabi sweep: 5ec139ddb86619001bbebb9e
Job Status: job has successfully run
Errors: None

Running Cavity Dynamics from 0.1us to 4.849999999999999us, in steps of 0.25us
Drive Amplitude = 0.061224489795918366
rabi sweep: 5ec13b66b86619001bbebbad
Job Status: job has successfully run
Errors

rabi sweep: 5ec16bd183f02c001a45effa
Job Status: job has successfully run
Errors: None

Running Cavity Dynamics from 0.1us to 4.849999999999999us, in steps of 0.25us
Drive Amplitude = 0.6275510204081632
rabi sweep: 5ec16bff7951d9001de1d2c2
Job Status: job has successfully run
Errors: None

Running Cavity Dynamics from 0.1us to 4.849999999999999us, in steps of 0.25us
Drive Amplitude = 0.6428571428571428
rabi sweep: 5ec16c2b83f02c001a45f001
Job Status: job has successfully run
Errors: None

Running Cavity Dynamics from 0.1us to 4.849999999999999us, in steps of 0.25us
Drive Amplitude = 0.6581632653061225
rabi sweep: 5ec16c5a7951d9001de1d2c5
Job Status: job has successfully run
Errors: None

Running Cavity Dynamics from 0.1us to 4.849999999999999us, in steps of 0.25us
Drive Amplitude = 0.673469387755102
rabi sweep: 5ec16c87b86619001bbebeed
Job Status: job has successfully run
Errors: None

Running Cavity Dynamics from 0.1us to 4.849999999999999us, in steps of 0.25us
Drive Amplitude = 0.688

## 4. Save the Data

In [11]:
import csv
import sys

fname = "rabiDrives_driveFreq{:0.2f}GHz_measFreq{:0.2f}GHz.csv".format(driveFreq/GHz, measFreq/GHz)
with open(fname, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    times = np.arange(0.1, 5.0, 0.25)
    heading = ['drive amp']
    for t in times:
        heading.append('{:0.2f}us'.format(t))
    
    writer.writerow(heading)
     
    for i in range(len(driveAmps)):
        row = [driveAmps[i]]
        for j in range(len(times)):
                row.append(all_readouts[i][j])
        writer.writerow(row)